In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing import image
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pynput
import time#暫停時間等功能
import threading#多線程處理
import keyboard#檢測鍵盤按鍵
from PIL import ImageGrab#抓取螢幕截圖
import uuid#隨機生成唯一ID

In [ ]:

dataset_1_file = 'dataset/1'
dataset_0_file = 'dataset/0'
if not os.path.exists(dataset_1_file):
        os.makedirs(dataset_1_file)
if not os.path.exists(dataset_0_file):
        os.makedirs(dataset_0_file)
stop_event = threading.Event()

def auto_take_screenshot():
    while not stop_event.is_set():
        data_name = os.path.join(dataset_0_file, f'0,{uuid.uuid4().hex}.png')
        img = ImageGrab.grab((550, 500, 750, 600))  # 擷取螢幕指定區域
        img.save(data_name)
        time.sleep(0.2)  # 每秒截圖一次
# 定義鍵盤監聽事件
def on_press(key):
    if key == pynput.keyboard.Key.shift:
        stop_event.set()  # 設置停止事件
        return False
    if key == pynput.keyboard.Key.space:
        data_name = os.path.join(dataset_1_file, f'1,{uuid.uuid4().hex}.png')
        img = ImageGrab.grab((550, 500, 750, 600))  # 擷取螢幕指定區域
        img.save(data_name)

# 啟動截圖子進程
auto_screenshot_process = threading.Thread(target=auto_take_screenshot)
#auto_screenshot_process.start()

# 監聽鍵盤事件 
with pynput.keyboard.Listener(on_press=on_press) as listener:
    listener.join()
  

In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory('dataset', image_size=(100, 200),validation_split=0.2, subset='training',seed=123)
validation_dataset = keras.preprocessing.image_dataset_from_directory('dataset', image_size=(100, 200),validation_split=0.2, subset='validation',seed=123)
train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))  # 正規化處理
validation_dataset = validation_dataset.map(lambda x, y: (x / 255.0, y))  # 正規化處理

In [ ]:
model = keras.Sequential()
model.add(layers.Input(shape=(100, 200,3)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset,epochs=50, validation_data=validation_dataset)

In [ ]:
test_image = keras.preprocessing.image.load_img('dataset/test4.png', target_size=(100, 200))
test_image = keras.preprocessing.image.img_to_array(test_image)
test_image = test_image / 255.0  # 正規化處理
test_image = np.expand_dims(test_image, axis=0)
model.predict(test_image)

In [ ]:
model = keras.models.load_model('jumpChrome.keras')  # 載入模型

In [ ]:
stop_event = threading.Event()
def AI_screenshot():
    while not stop_event.is_set():
        img = ImageGrab.grab((550, 500, 750, 600))  # 擷取螢幕指定區域
        img = keras.preprocessing.image.img_to_array(img)
        img = img / 255.0  # 正規化處理
        press = model.predict(np.expand_dims(img, axis=0))[0][0]
        print(press)
        if press > 0.5:
            keyboard.press('space')
        time.sleep(0.01)  # 每間隔截圖一次
# 定義鍵盤監聽事件
def on_press(key):
    if key == pynput.keyboard.Key.shift:
        stop_event.set()  # 設置停止事件
        return False
# 啟動截圖子進程
AI_process = threading.Thread(target=AI_screenshot)
AI_process.start()           
# 監聽鍵盤事件
with pynput.keyboard.Listener(on_press=on_press) as listener:
    listener.join()